[View in Colaboratory](https://colab.research.google.com/github/amohamdy/web-task/blob/master/tf_learn.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


**Make Current Directory**

In [0]:
import os
os.chdir("drive/My Drive/sentiment-model/tf-learn")

In [3]:
!ls

tf-learn.ipynb	tweets-pos-neg.csv


**Download and unzip ngrok**

In [4]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


Redirecting output to ‘wget-log’.
Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


**Run Tensor Board**

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

**Run ngrok**

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

**Get URL**
Run the next cell to start the training before open the url.

In [7]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://33b28690.ngrok.io


In [8]:
!pip install tflearn

    100% |████████████████████████████████| 102kB 6.1MB/s 
  Running setup.py bdist_wheel for tflearn ... - \ done
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [9]:
import os   # iterate throw DIRECTORIES 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import set_random_seed
tf.set_random_seed(7)
from nltk.tokenize import word_tokenize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.layers.core import input_data, dropout, fully_connected, flatten
from tflearn.layers.conv import conv_1d,max_pool_1d
from tflearn.layers.estimator import regression
import re
import string
import collections
import time
from sklearn.utils.multiclass import type_of_target

Using TensorFlow backend.


**Load CSV File**

In [0]:
#Read CSV Files 
df=pd.read_csv('tweets-pos-neg.csv', usecols = ['text','airline_sentiment'])
df = df.reindex(['text','airline_sentiment'], axis=1) #reorder columns
df=df.apply(lambda x: x.astype(str).str.lower()) #convert data to lower case

**Normalization Process**

In [0]:
def normalize(text):
    text= re.sub(r"http\S+", r'', text) #replaces links with "URL" or space
    text= re.sub(r"@\S+", r'', text) #replaces @Mention with "mention" or space
    punctuation = re.compile(r'[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~|0-9]')
    text = re.sub(punctuation, ' ', text)
    text= re.sub(r'(.)\1\1+', r'\1', text)  #solve el.ongation problem ex loooool=>lol, lool=>lool  
    return text

In [0]:
def prepareDataSets(df):
    sentences=[]
    for index, r in df.iterrows():
        text= normalize(r['text'])
#         if r['airline_sentiment']=='positive':
#             sentences.append([text,1])
#         else:
#             sentences.append([text,0])
#         #text= stopWordRemove(text)
        sentences.append([text,r['airline_sentiment']])
        df_sentences=pd.DataFrame(sentences,columns=['text','airline_sentiment'])
    return df_sentences

In [0]:
edit_df=prepareDataSets(df)
edit_df=shuffle(edit_df)


In [0]:
X=edit_df.iloc[:,0]
Y=edit_df.iloc[:,1]

** Convert reviews to Tokens then Tokens to Integers**

In [0]:

max_features = 50000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X.values)
#convert review tokens to integers
X_seq = tokenizer.texts_to_sequences(X)

In [16]:
print('Fitted tokenizer on {} documents'.format(tokenizer.document_count))
print('{} words in dictionary'.format(tokenizer.num_words))
print('Top 5 most common words are:', collections.Counter(tokenizer.word_counts).most_common(5))

Fitted tokenizer on 11541 documents
50000 words in dictionary
Top 5 most common words are: [('to', 6988), ('the', 5085), ('i', 4235), ('a', 3763), ('you', 3403)]


**Get The Length of each Review then get MAX **

In [0]:
def countWords(review):
    counts=[]
    for review in X:
        review_legnth=len(review.split())
        counts.append(review_legnth)
    return counts

In [18]:
counts=countWords(X)
max(counts)

32

In [0]:
# Equal Length of Sequence Padding Sequence according to MAX length of reviews
seq_len=35
X_pad = pad_sequences(X_seq,maxlen=seq_len)

In [0]:
# Encoding Target Value
# convert target value from string to integer
le=LabelEncoder()
Y_le=le.fit_transform(Y)
Y_le_oh=to_categorical(Y_le, nb_classes=2)

In [21]:
Y_le_oh
print(type_of_target(Y_le_oh))

multilabel-indicator


**Train, Test split**

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X_pad,Y_le_oh, test_size = 0.33, random_state = 42)
X_train, X_Val, Y_train, Y_Val = train_test_split(X_train,Y_train, test_size = 0.1, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(X_Val.shape,Y_Val.shape)

(6958, 35) (6958, 2)
(3809, 35) (3809, 2)
(774, 35) (774, 2)


In [0]:
# Create the model
tf.reset_default_graph()     #solve problem list out of index
net =input_data(shape=[None, 35], name='input')   
net=tflearn.embedding(net, max_features, output_dim=32)
conv1=conv_1d(net, 100, 3, padding='same', activation='relu')
conv1=max_pool_1d(conv1,2)
conv1=flatten(conv1)
conv1=fully_connected(conv1, 256, activation='relu')
conv1=fully_connected(conv1, 2, activation='softmax')
conv1 = regression(conv1, optimizer='adam', learning_rate=0.001,
                     loss='categorical_crossentropy', name='target')
Name="tflearn-{}".format(int(time.time()))
model = tflearn.DNN(conv1, tensorboard_verbose=3, tensorboard_dir='./log/{}'.format(Name))

#print(conv1.summary())

In [0]:
# Name="Sentimnt-Analysis-tflearn-{}".format(int(time.time()))
#tensorboard = TensorBoard(log_dir='./log/{}'.format(Name))

In [84]:
history=model.fit(X_train, Y_train, n_epoch=3, validation_set=(X_Val, Y_Val),show_metric=True, batch_size=32)
scores = model.evaluate(X_test, Y_test)
print("Accuracy: %.2f%%" % (scores[-1]*100))

Training Step: 653  | total loss: 0.04374 | time: 31.976s
| Adam | epoch: 003 | loss: 0.04374 - acc: 0.9904 -- iter: 6944/6958
Training Step: 654  | total loss: 0.04392 | time: 33.139s
| Adam | epoch: 003 | loss: 0.04392 - acc: 0.9913 | val_loss: 0.21007 - val_acc: 0.9315 -- iter: 6958/6958
--
Accuracy: 91.60%


In [0]:
def tflearn_evaluate(): 
    # predict class with test set
    Y_predict =model.predict_label(X_test)
    #Y_predict=tf.nn.softmax(Y_predict)
    print('Accuracy:\t{:0.1f}%'.format(accuracy_score(tf.argmax(Y_test, axis=1),Y_predict)*100))
    
    #classification report
    print('\n')
    print(classification_report(Y_test, Y_predict))
    #confusion matrix
    confmat = tf.confusion_matrix(Y_test, Y_predict,num_classes=None,dtype=tf.int32,name=None,weights=None)

#     fig, ax = plt.subplots(figsize=(4, 4))
#     ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
#     for i in range(confmat.shape[0]):
#         for j in range(confmat.shape[1]):
#             ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
#     plt.xlabel('Predicted label')
#     plt.ylabel('True label')
#     plt.tight_layout()

In [81]:
tflearn_evaluate()

ValueError: ignored

In [0]:
Y_predict =model.predict(X_test)
Y_predict=tf.nn.softmax(Y_predict)
b=tf.equal(tf.argmax(Y_test,axis=None),tf.argmax(Y_predict,axis=None))

accuracy=tf.reduce_mean(tf.cast(b,'float'))

In [73]:
accuracy

<tf.Tensor 'Mean_1:0' shape=() dtype=float32>